# Dealing with large datasets


In [3]:
import pandas as pd
from datetime import datetime
import os
import subprocess

## Download and process static data
This Section of the notebook 
1. downloads static data sources, 
2. loads them into memory, 
3. cleans that data then saves a cleaned data product, which may be used lateron for some analysis.

### Download files from on-line host.
Including this step in the pipeline ensures that the data processing steps are reproducible. This will save the severe headache that comes from trying to share particular cleaned data products.

The script below will:

1. Check if the data directory exists and clear it if it does.
2. Create a new data directory.
3. Download a list of specified data files from an online source into the directory.

This approach ensures that everyone working on this project has access to the same data in its original form, facilitating consistent results across different environments.

We will use Python for all operations, including file handling and downloading data, for simplicity and efficiency.

In [4]:
# Define the directory where data will be stored
DATA_DIR = "../data/camels/"

# Remove the data directory if it already exists to ensure a fresh start
if os.path.exists(DATA_DIR):
    subprocess.run(['rm', '-Rf', DATA_DIR])

# Create a new data directory
os.makedirs(DATA_DIR)

# List of filenames to be downloaded
filenames = [
    "camels_clim.txt", "camels_geol.txt", "camels_hydro.txt",
    "camels_name.txt", "camels_soil.txt", "camels_topo.txt", "camels_vege.txt"
]

# Loop through each file and download it to the data directory
for filename in filenames:
    url = f"https://gdex.ucar.edu/dataset/camels/file/{filename}"
    output_path = os.path.join(DATA_DIR, filename)
    subprocess.run(['wget', '-O', output_path, url])

--2024-06-12 03:00:12--  https://gdex.ucar.edu/dataset/camels/file/camels_clim.txt
Resolving gdex.ucar.edu (gdex.ucar.edu)... 128.117.181.6
Connecting to gdex.ucar.edu (gdex.ucar.edu)|128.117.181.6|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 100673 (98K) [text/plain]
Saving to: ‘../data/camels/camels_clim.txt’

     0K .......... .......... .......... .......... .......... 50% 3.68M 0s
    50K .......... .......... .......... .......... ........  100% 3.53M=0.03s

2024-06-12 03:00:13 (3.60 MB/s) - ‘../data/camels/camels_clim.txt’ saved [100673/100673]

--2024-06-12 03:00:13--  https://gdex.ucar.edu/dataset/camels/file/camels_geol.txt
Resolving gdex.ucar.edu (gdex.ucar.edu)... 128.117.181.6
Connecting to gdex.ucar.edu (gdex.ucar.edu)|128.117.181.6|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 71583 (70K) [text/plain]
Saving to: ‘../data/camels/camels_geol.txt’

     0K .......... .......... .......... .......... .......... 71% 3.68M 

### Share the Processing Code, Not Just the Processed Data

Sharing the code used for data processing, rather than just the processed data, is crucial for ensuring reproducibility, especially in data science. While this approach might become challenging with very large datasets, it's essential for maintaining transparency and allowing others to understand and replicate your workflow. By sharing code, you provide insights into how raw data is transformed, cleaned, and made ready for analysis, which is invaluable for collaborative projects and scientific research.


In [5]:
# List of filenames to be loaded
filenames = ["camels_clim.txt", "camels_geol.txt", "camels_hydro.txt",
             "camels_name.txt", "camels_soil.txt", "camels_topo.txt", "camels_vege.txt"]

# Dictionary to store DataFrames for each file
dfs = {}

# Loop through each file, read it into a DataFrame, and store it in the dictionary
for filename in filenames:
    with open(f"../data/camels/{filename}", "r") as f:
        # Read the file using pandas, with ';' as the separator and 'gauge_id' as the index column
        dfs[filename] = pd.read_csv(f, sep=";", index_col="gauge_id")

# Concatenate all DataFrames along the columns
df = pd.concat([dfs[filename] for filename in filenames], axis=1)

### Handling Text Data and NaN Values

Once we consolidate all our data into a single DataFrame, we often encounter text data and NaN (Not a Number) values. Depending on our analysis goals, these may not be useful. For our current example, we require a dataset with complete information, meaning no missing values (NaNs), and our analysis will focus on continuous numerical data. Therefore, it's important to identify and appropriately handle text and NaN values to prepare our dataset for further analysis.

Let's start by taking a preliminary look at our DataFrame to understand its structure and the nature of the data it contains.


In [6]:
# Display the first few rows of the DataFrame to inspect its contents
df.head()

,p_mean,pet_mean,p_seasonality,frac_snow,aridity,high_prec_freq,high_prec_dur,high_prec_timing,low_prec_freq,low_prec_dur,...,area_geospa_fabric,frac_forest,lai_max,lai_diff,gvf_max,gvf_diff,dom_land_cover_frac,dom_land_cover,root_depth_50,root_depth_99
gauge_id,,,,,,,,,,,,,,,,,,,,,
1013500,3.126679,1.971555,0.187940,0.313440,0.630559,12.95,1.348958,son,202.20,3.427119,...,2303.95,0.9063,4.167304,3.340732,0.804567,0.371648,0.883452,Mixed Forests,NaN,NaN
1022500,3.608126,2.119256,-0.114530,0.245259,0.587356,20.55,1.205279,son,233.65,3.662226,...,620.38,0.9232,4.871392,3.746692,0.863936,0.337712,0.820493,Mixed Forests,0.237435,2.238444
1030500,3.274405,2.043594,0.047358,0.277018,0.624111,17.15,1.207746,son,215.60,3.514262,...,3676.09,0.8782,4.685200,3.665543,0.858502,0.351393,0.975258,Mixed Forests,NaN,NaN
1031500,3.522957,2.071324,0.104091,0.291836,0.587950,18.90,1.148936,son,227.35,3.473644,...,766.53,0.9548,4.903259,3.990843,0.870668,0.398619,1.000000,Mixed Forests,0.250000,2.400000
1047000,3.323146,2.090024,0.147776,0.280118,0.628929,20.10,1.165217,son,235.90,3.691706,...,904.94,0.9906,5.086811,4.300978,0.891383,0.445473,0.850450,Mixed Forests,0.241027,2.340180


### Cleaning the Data for Analysis

In data analysis, the treatment of NaN (Not a Number) values depends on the context and objectives of your study. While NaNs can be acceptable or even meaningful in certain scenarios, they might not be suitable for others. In our hypothetical analysis, we require a dataset without any missing values. Therefore, we will remove columns containing NaN values to ensure our dataset is complete and ready for analysis.

This step is crucial for maintaining the integrity and reliability of our analysis, as missing data can lead to biased or inaccurate results.


In [7]:
# Remove columns with any NaN values from the DataFrame
df = df.dropna(axis=1)

### Further Data Cleaning: Removing One Hot Encoded Data

In many analytical scenarios, categorical data represented as strings or One Hot encoded data can be quite useful. However, for our specific analysis, we need a dataset consisting solely of numerical values. Therefore, we will identify and remove columns that contain string data, which often represent categorical variables.

This step is crucial for aligning our dataset with the requirements of our analysis, ensuring that the data is in the correct format for the statistical or machine learning methods we plan to apply.


In [8]:
# Initialize a list to hold the names of columns to be dropped
drop_these_columns = []

# Iterate over each column in the DataFrame
for camels_data_column in df.columns.values:
    # Check if the first value in the column is of string type
    if type(df[camels_data_column].values[0]) == str:
        # If it is a string, add the column name to the list
        drop_these_columns.append(camels_data_column)

# Drop the identified columns from the DataFrame
df = df.drop(drop_these_columns, axis=1)


### Saving the Cleaned Data with a Unique Filename

When sharing and managing data files, especially in a collaborative environment, it's good to avoid confusion caused by multiple versions of the same file. To prevent issues related to version control and ensure traceability, we'll save our cleaned dataset with a unique and descriptive filename. This filename will include the current date and time, along with the initials of the person who processed the data. Such a naming convention makes it easier to track changes over time and understand the lineage of the dataset.


In [9]:
# Generate a timestamp string for the current date and time
nowstring = datetime.today().strftime("%d-%m-%Y_%H%M")

# Initials of the data processor (change as needed)
creator_initials = "jf"

# Save the DataFrame to a CSV file with a unique name
df.to_csv(f"../data/camels/camels_attributes_cleaned_{nowstring}_{creator_initials}.csv")